'----------------------------------------------------------------------------------------------------

# Module 9 Part 1: Intorudction to TensorFlow and Neural Networks

- 1 Introduction
- 2 Learning Objectives
- 3 Reading and Resources
- 4 TensorFlow Graphs
    - 4.1 Overview of graphs
    - 4.2 Building your own graphs
    - 4.3 Managing graphs
- 5 Linear Regression with TF
    - 5.1 Implementing gradient gescent
    - 5.2 Manually computing gradients
    - 5.3 Using Autodiff
- 6 TF Optimizers
    - 
- 7 MiniBatch

# Introduction

Originally developed by researchers and engineers from the Google Brain team, [**TensorFlow**](https://www.tensorflow.org/) is a very powerful open source library for high performance numerical computation. It is built using highly optimized C++ code and computational graph arhcitecture (which we will cover shortly), allowing for high performance speeds and efficient paraliization that make it extrmely well suited for large scale machine learning applications. Tensorflow has quickly become to tool of choice for many machine learning and deep learning engineers, and is the backbone for popular high level neural network APIs like Keras. 

There are many reasons tensorflow has become as popular as it is today, but the most important of these is probably the ease with which operations can be parellelized and distributed across multiple CPU or GPUs. This allows the model developer to train massive neural networks (which can have million of parameters) on humongous training sets in a reasonable amount of time by splitting the computations across hundreds of servers.

# Learning Outcomes

In this module, you will:

* Become familiar with tensorflow api and it's underlying architecture
* Build a simple model in a tensorflow graph and learn how to perform training with various optimizers
* Learn how to save model checkpoints and visualize your model's results using tensorboard

# Readings and Resources

The majority of the notebook content borrows from the recommended readings. We invite you to further supplement this notebook with the following recommended texts:

Python Community (2018). *The Python Tutorial*. Retrieved from the following [web page](https://docs.python.org/3/tutorial/index.html).

Kuchling A.M. (2018). *Functional Programming HOWTO*. Retrieved from the following Python documentation [web page](https://docs.python.org/3.6/howto/functional.html).

https://jacobbuckman.com/post/tensorflow-the-confusing-parts-1/


# Graphs & Sessions

## Graphs

Tensorflow's fundemental architecture is based on a certain type of computational abstraction known as a “computation graph”. A copmutational graph, or dataflow graph, essentially represents the computations the user defines in terms of the dependencies between individual tensorflow operations (often called 'ops'). A dataflow graph is composed of two main compoenents:
- 1) **Nodes:** these represent units of computation and are typically stored as a set of `tf.Operation` objects (which are basically mathematical operations)
- 2) **Edges:** these represent the data consumed or produced by a computation (the input or output of a tf.Operation), and are stored as `tf.Tensor` objects. 

For example, in a TensorFlow graph, the `tf.matmul` operation would correspond to a single node with two incoming edges (the matrices to be multiplied) and one outgoing edge (the result of the multiplication). The image below represents the graph of a very simple feedforward neural network (which we will cover shortly).  
![](images/c4_m9_p1_tensors_flowing.gif)

Together, nodes and edges make up the overall **Graph structure,** indicating how individual operations are composed together, **but NOT prescribing how they should be used.** The graph structure is like assembly code: inspecting it can convey some useful information, but it does not contain all of the useful context that source code conveys.

When we manipulate Tensorflow with Python, the first thing we do with our Python code is **assemble the computation graph**. Once that is done, the second thing we do is to interact with it (using Tensorflow’s “sessions”, which we will cover shortly). If this sounds confusing, don't worry, it will become clearer once we dive into the code - which we will start doing now! Let's begin by builing a very simple graph. We will start by using three simple operations:
- `tf.constant()`: this simply adds a 'constant value' tensor to the graph (a value that does not change)
- `tf.variable`: this adds a variable tensor which can be changed based on the calculations in the computation graph (for instance, you would assign the coefficients in a linear regression as a tf.Variable)

*NOTE: It’s important to keep in mind that the computation graph does **NOT live inside of your variables; it lives in the global namespace.** When you assign a tensorflow object to a variable in python, that variable merely becomes a pointer to the tensorflow object in the global namespace. You don't have to worry about this for now, but this becomes an important technical detail when working with complicated models and distributed computing systems.*





In [53]:
# First import tensorflow as tf
import tensorflow as tf

# Add variables to the graph
x = tf.Variable(3, name='x')
y = tf.constant(4, name='y')

# Join x and y in the graph
f = x*x*y+y+2
print(f)

Tensor("add_3:0", shape=(), dtype=int32)


It is also important to note that **every time we call tf.constant, we create a new node in the graph.** This is true even if the node is functionally identical to an existing node, even if we re-assign a node to the same variable, or even if we don’t assign it to a variable at all.

In [12]:
for i in range(0,5):
    tf.constant(2, name='repeat')

In [33]:
tf.get_default_graph().get_all_collection_keys()

['variables', 'trainable_variables']

In [49]:
tf.reset_default_graph()

In [52]:
?tf.get_default_graph().as_graph_def().

Object `tf.get_default_graph().as_graph_def().` not found.


In [41]:
[n.name for n in tf.get_default_graph().as_graph_def().node]


['x', 'y', 'mul', 'mul_1', 'add', 'add_1/y', 'add_1']

In [21]:
tf.get_collection('trainable_variables')

[<tf.Variable 'x:0' shape=() dtype=int32_ref>,
 <tf.Variable 'y:0' shape=() dtype=int32_ref>,
 <tf.Variable 'y_1:0' shape=() dtype=int32_ref>]

Note


Every time we call tf.constant, we create a new node in the graph. This is true even if the node is functionally identical to an existing node, even if we re-assign a node to the same variable, or even if we don’t assign it to a variable at all.



## Sessions

## Overview

The role of the session is to handle the memory allocation and optimization that allows us to actually perform the computations specified by a graph. You can think of the computation graph as a “template” for the computations we want to do: it lays out all the steps. In order to make use of the graph, we also need to make a session, which allows us to actually do things; for example, going through the template node-by-node to allocate a bunch of memory for storing computation outputs. In order to do any computation with Tensorflow, you need both a graph and a session.

The session contains a pointer to the global graph, which is constantly updated with pointers to all nodes. That means it doesn’t really matter whether you create the session before or after you create the nodes. 2

After creating your session object, you can use sess.run(node) to return the value of a node, and Tensorflow performs all computations necessary to determine that value.



Note that a `tf.Tensor` object does not hold any values (like, for instance, a numpy array). It simply represents one of the outputs of a `tf.Operation`, and provides a means of computing those values in a TensorFlow `tf.Session`.

This class has two primary purposes:

A Tensor can be passed as an input to another Operation. This builds a dataflow connection between operations, which enables TensorFlow to execute an entire Graph that represents a large, multi-step computation.

After the graph has been launched in a session, the value of the Tensor can be computed by passing it to tf.Session.run. t.eval() is a shortcut for calling tf.get_default_session().run(t).


## Why use graphs?



- **Graph collections:** TensorFlow provides a general mechanism for storing collections of metadata in a `tf.Graph`. The `tf.add_to_collection` function enables you to associate a list of objects with a **key** (where `tf.GraphKeys` defines some of the standard keys), and `tf.get_collection` enables you to look up all objects associated with a key. Many parts of the TensorFlow library use this facility: for example, when you create a `tf.Variable`, it is added by default to collections representing "global variables" and "trainable variables". When you later come to create a `tf.train.Saver` or `tf.train.Optimizer`, the variables in these collections are used as the default arguments.




Dataflow has several advantages that TensorFlow leverages when executing your programs:
- **Parallelism:** By using explicit edges to represent dependencies between operations, it is easy for the system to identify operations that can execute in parallel (crucial for complex models like neural networks).
- **Distributed execution:** By using explicit edges to represent the values that flow between operations, it is possible for TensorFlow to partition your program across multiple devices (CPUs, GPUs, and TPUs) attached to different machines. TensorFlow automatically inserts the necessary communication and coordination between devices, making the process fairly straightforward. 
- **Compilation:** TensorFlow's XLA compiler can use the information in your dataflow graph to generate faster code, for example, by fusing together adjacent operations.
- **Portability:** The dataflow graph is a language-independent representation of the code in your model. You can build a dataflow graph in Python, store it in a SavedModel, and restore it in a C++ program for low-latency inference (super fast predictions!).


## What is a tf.Graph?

A tf.Graph contains two relevant kinds of information:





## Building a tf.Graph
Most TensorFlow programs start with a dataflow graph construction phase. In this phase, you invoke TensorFlow API functions that construct new tf.Operation (node) and tf.Tensor (edge) objects and add them to a tf.Graph instance. TensorFlow provides a default graph that is an implicit argument to all API functions in the same context. For example:

Calling tf.constant(42.0) creates a single tf.Operation that produces the value 42.0, adds it to the default graph, and returns a tf.Tensor that represents the value of the constant.

Calling tf.matmul(x, y) creates a single tf.Operation that multiplies the values of tf.Tensor objects x and y, adds it to the default graph, and returns a tf.Tensor that represents the result of the multiplication.

Executing v = tf.Variable(0) adds to the graph a tf.Operation that will store a writeable tensor value that persists between tf.Session.run calls. The tf.Variable object wraps this operation, and can be used like a tensor, which will read the current value of the stored value. The tf.Variable object also has methods such as tf.Variable.assign and tf.Variable.assign_add that create tf.Operation objects that, when executed, update the stored value. (See Variables for more information about variables.)

Calling tf.train.Optimizer.minimize will add operations and tensors to the default graph that calculates gradients, and return a tf.Operation that, when run, will apply those gradients to a set of variables.

Most programs rely solely on the default graph. However, see Dealing with multiple graphs for more advanced use cases. High-level APIs such as the tf.estimator.Estimator API manage the default graph on your behalf, and--for example--may create different graphs for training and evaluation.

Note: Calling most functions in the TensorFlow API merely adds operations and tensors to the default graph, but does not perform the actual computation. Instead, you compose these functions until you have a tf.Tensor or tf.Operation that represents the overall computation--such as performing one step of gradient descent--and then pass that object to a tf.Session to perform the computation. See the section "Executing a graph in a tf.Session" for more details.


NOTE: This guide will be most useful if you intend to use the low-level programming model directly. Higher-level APIs such as tf.estimator.Estimator and Keras hide the details of graphs and sessions from the end user, but this guide may also be useful if you want to understand how these APIs are implemented.


## Building your own graph

USE THIS:
https://jacobbuckman.com/post/tensorflow-the-confusing-parts-1/

A default Graph is always registered, and accessible by calling tf.get_default_graph. To add an operation to the default graph, simply call one of the functions that defines a new Operation:

c = tf.constant(4.0)
assert c.graph is tf.get_default_graph()
Another typical usage involves the tf.Graph.as_default context manager, which overrides the current default graph for the lifetime of the context:

g = tf.Graph()
with g.as_default():
  # Define operations and tensors in `g`.
  c = tf.constant(30.0)
  assert c.graph is g
Important note: This clas

# Session

## Overview

The role of the session is to handle the memory allocation and optimization that allows us to actually perform the computations specified by a graph. You can think of the computation graph as a “template” for the computations we want to do: it lays out all the steps. In order to make use of the graph, we also need to make a session, which allows us to actually do things; for example, going through the template node-by-node to allocate a bunch of memory for storing computation outputs. In order to do any computation with Tensorflow, you need both a graph and a session.

The session contains a pointer to the global graph, which is constantly updated with pointers to all nodes. That means it doesn’t really matter whether you create the session before or after you create the nodes. 2

After creating your session object, you can use sess.run(node) to return the value of a node, and Tensorflow performs all computations necessary to determine that value.

## Running the session

....

Object-oriented programming (OOP) refers a programming language model which defines "objects" containing data and functions that determine what types of operations can be applied to the data. If you are new to OOP, you can familiarize yourselves with the basics by reviewing the following web page on Wikipedia for general understanding of terms and concepts: [Object-Oriented Programming](https://en.wikipedia.org/wiki/Object-oriented_programming). If you are looking for a book explaining the concepts of object-oriented software development, you can get a copy of the classic book ["Design Patterns: Elements of Reusable Object-Oriented Software"](https://www.amazon.ca/Design-Patterns-Elements-Reusable-Object-Oriented/dp/0201633612/ref=sr_1_1?s=books&ie=UTF8&qid=1533523480&sr=1-1&dpID=51szD9HC9pL&preST=_SX198_BO1,204,203,200_QL40_&dpSrc=srch) by Erich Gamma, Richard Helm, Ralph Johnson, and John Vlissides.

At the core of the OOP are the concepts of a **class** which defines the data format and available procedures of an **object**. Any object is an instance of a corresponding class.

Here is one of the definitions of classes and objects:
> Classes – the definitions for the data format and available procedures for a given type or class of object; may also contain data and procedures (known as class methods) themselves, i.e. classes contain the data members and member functions.

>Objects – instances of classes

Think of a class as a template or a blueprint for any object created of this class.

We will use a simple example to explain the concept and components of OOP without writing any code first. For this initial example, we will define a **class Vehicle**. This class can define the following *fields* of a generalized vehicle: 
- model, 
- year, 
- colour, 
- type, 
- transmission type.

The class would also contain definitions of *functionality* of a vehicle, for example:
- EngineStart(), 
- accelerate(), 
- EngineStop(), 
- turn() and others. 

These functions are called **methods**. Together, fields and methods are called **attributes** of the class.

Using this **class**, we can create multiple **objects** of a class Vehicle which will have different combinations and values of the attributes defined by the class. 

When a new object is created, we say that we create an instance of a class, or **instantiate** a new object.

### Classes and objects in Python

In the next example we will create a class **Book**. It will accept two parameters, an author of the book and its title. The class will have a method which will print out the book description as a single sentence listing the title and author of the book.

A class in Python is created using the keyword `class`, then we type the name of the class we want to create, and a colon (:). The class body is indented (usual 4 spaces).

Here is the definition of a class `Book`:

In [13]:
class Book:
    def __init__(self, author, title):
        self.author = author
        self.title = title
    def bookdesc(self):
        return "Book '{}' by {}".format(self.title, self.author)

The class `Book` defines two methods, `__init__()` and `bookdesc()`. 

The name of the method `__init__()` starts and ends with double underscores (often called "dunder"), it is one of the so-called magic methods. This method is called automatically every time the new instance of a class is created. This method is call a **constructor** and it **initiates** fields of the class. In our example, method `__init__()` has 3 arguments. The argument `self` is always the first argument of a method `__init__()` and it refers to the object itself, it helps Python to keep track of different instances of the class.

The other two arguments of a method `__init__()` are the author name and title of a book. The function will **initialize** the new object by assigning initial values to the object.

The second method, `bookdesc()`, will print out the name and title of the book. It also takes `self` as an argument.
Let's create some books:

In [16]:
bookA = Book("George R.R. Martin", "A Game of Thrones")

Let's explain what Python did when the above line of code was executed:

1. Python interpreter has created a new instance of a class Book
2. Method `__init__()` was executed automatically. It used the string `"George R.R. Martin"` as the value of parameter `author` and string `"A Game of Thrones"` as a value of parameter `title` for the method `__init__()`. The code within the function initialized attributes `author` and `title` of a new object. These are called **instance attributes**.
3. A new object is assigned to a variable `bookA`. This object has two attributes, `author` and `title`.
4. The new object has a method `bookdesc()` which will print out the title and author of a book. This method is called **instance method**.

We can check the type of an object:

In [17]:
type(bookA)

__main__.Book

Variable `bookA` points to an object of class `Book`.

We can check the values of the `title` and `author` instance attributes of this object:

In [18]:
bookA.author

'George R.R. Martin'

In [19]:
bookA.title

'A Game of Thrones'

Calling instance method `bookdesc()`:

In [20]:
bookA.bookdesc()

"Book 'A Game of Thrones' by George R.R. Martin"

More information about Python classes can be found in the Python documentation, [Chapter 9, Classes.](https://docs.python.org/3.3/tutorial/classes.html) in The Python Tutorial.

### Characteristics of Python objects

Each object in Python has three characteristics:

* Object type, 
* Object value,
* Object identity.

**Object type** tells Python what kind of an object it's dealing with. A type could be a string, or a list, or any other type of Python object, including custom-defined objects, like we saw above when we checked the type of the object that the variable `bookA` is pointing to.

**Object value** is the data value contained by the object. In the example above, it was the title of the book and name of an author. For an integer, for example, it can be the value of a number.

**Object identity** is an identity number for the object. Each distinct object in the computer's memory will have its own identity number.

To check an identifier of an object, we can use method `id()`. It returns an integer which uniquely identifies an object:

In [21]:
id(bookA)

4529273488

The number above is an ID of an object which variable bookA is pointing to. Multiple variables can point to the same object. Let's illustrate this with a simple example. 

First, we will create a new list object `listobj` of two integers:

In [14]:
listobjA = [22, 33]

This object has a **type** and **id** which we can easily obtain using corresponding methods:

In [15]:
type(listobjA)

list

In [16]:
id(listobjA)

4445020488

We can create another variable, `listobjB`, which will point to the same list object `[22, 33]`:

In [17]:
listobjB = listobjA
listobjB

[22, 33]

We can easily confirm that `listobjB` points to the same list by validating it's `id()`:

In [18]:
id(listobjB)

4445020488

And just as we expected, the **`ID`** of `listobjA` and `listobjB` are exactly the same. Another way of confirming that `listobjA` and `listobjB` are pointing to the same object is to use the `is` operator:

In [19]:
listobjA is listobjB

True

We can also validate the *equality* of these two objects by using the operator `==` :

In [20]:
listobjA == listobjB

True

And we can see from the above that `listobjA` and `listobjB` variables have the same value and are pointing to the same list object. 

Before we continue, let's create another list object, name it `listobjC` and it will be a copy of `listobjA`. 

In [21]:
listobjC = list(listobjA)

In [22]:
# We can validate that it has the same value:

listobjC

[22, 33]

In [23]:
# but different ID:

id(listobjC)

4445235144

In [24]:
listobjC is listobjA

False

In [25]:
# even though they are equal:

listobjC == listobjA

True

Now let's add a third integer, `44`, to the end of the original list object using `append()` function: 

In [26]:
listobjA.append(44)
listobjA

[22, 33, 44]

In [27]:
# The ID of this object didn't change, it is the same object as above:

id(listobjA)

4445020488

What happened to `listobjB`? If it was pointing to the same list object as `listobjA`, it should have changed as well. Let's check:

In [28]:
listobjB

[22, 33, 44]

And it changed indeed. But what about `listobjC` which we created as a copy of the original list?

In [29]:
listobjC

[22, 33]

`listobjC` didn't change because it is a separate object with its own ID.

This is a very important concept to remember as you are working with Python objects.

We also need to point out that each element within a list object is an object as well. The list object `listobjA` is a container of objects, elements of the list. In our case the elements are integers with individual IDs. Let's check the object ID of each element within a list:

In [30]:
id(listobjA[0])

4410624672

In [31]:
id(listobjA[1])

4410625024

In [32]:
id(listobjA[2])

4410625376

In [34]:
# Since listobjB is pointing to the same list object as listobjA, 
# the object ID of the last, third element, should be the same

id(listobjB[2])

4410625376

In [36]:
# List elements are of type `integer`
# whereas the list itself is of type `list`

print(type(listobjA[2]))
print(type(listobjA))

<class 'int'>
<class 'list'>


### **EXERCISE 1:** Explore Python objects

To visually demonstrate this important concept, we will use [**Python Tutor**](http://pythontutor.com/) website which helps to visualize what happens when each line of code is executed - [http://pythontutor.com/live.html#mode=edit](http://pythontutor.com/live.html#mode=edit). 

We will open the interactive session so that we can see the changes, and reproduce all the steps above. You are encouraged to open the link in a separate browser window and follow along.

**NOTE**: If you want to make sure that you see the same visualization as on the screen shots below, please set the following values for the three drop-down lists in the bottom of the screen:
- first drop-down: **"show exited frames (Python)"**
- second drop-down: **"render all objects on the heap (Python)"**
- third drop-down: **"draw pointers as arrows [default]"**. During the exercise, you can also select "use text labels for pointers" in this drop-down, it will allow you to see object ID instead of arrows.

Refer to the image below:

<img src='.\files\drop-downs.jpg' width="800" alt="Drop-down settings in Python Tutor window">

**Picture 1.** Drop-down settings in Python Tutor window

**Step 1 -** Create `listobjA` as a list object with two values: `[22, 33]`

<img src='.\files\step1.jpg' width="700" height="700" alt="List `listobjA` is created">

**Picture 2.** Create list object `listobjA`

As you can see in the image above, the list `listobjA` is a container of type `list` with two elements, both elements are integer objects. The visualization also shows indexes for the list elements within a list object. 

If we change the value of the third drop-down to "use text labels for pointers", we will see that the list object itself has an id = `id1` and the list elements have IDs `id2` and `id3`. The tool also shows each element of a list being an independent object of type `int`.

<img src='.\files\step1_2_.jpg' width="700" height="700" alt="List object `listobjA` with object IDs displayed">

**Picture 3.** List object `listobjA` with object IDs displayed

**NOTE:** IDs in the code above and IDs generated by the Python Tutor will be always different. For simplicity and demonstration purposes, Python Tutor IDs will start from `id=1` when you refresh the screen and start a new session.

We will change the value of the third drop-down back to "draw pointers as arrows [default]" and continue with the steps in our example.

** Step 2** - Create `listobjB` as follows `listobjB = listobjA` in the Python Tutor window. You should see the following result:

<img src='.\files\step2_createB.jpg'/ width="700" height="700" alt="Created `listobjB`">

**Picture 4.** Created `listobjB`

As we expected, Python simply created a new variable, `listobjB`, and pointed it to the same list object that `listobjA` is pointing to.

**Step 3** - In Python Tutor window, create `listobjC` as follows: `listobjC = list(listobjA)`.

Here is the result of this step:

<img src='./files/step3_createC_1.jpg/' width="700" height="700" alt="Create `listobjC`">

**Picture 5.** Create `listobjC`

We can note an interesting detail - Python will create a new list object for `listobjC` but it will point to the same list elements, integer objects `22` and `33`. If we switch into "use text labels for pointers" view, we can see that an ID for `listobjC` is `id4`:

<img src='.\files\step3_createC_2.jpg'/ width="700" height="700" alt="View object IDs">


**Picture 6.** View object IDs for all objects


Switching back into the arrow view for next step.

**Step 4 -** Add an element with value `44` to the end of `listobjA`

<img src='.\files\step4_add44toA.jpg' width="700" height="700" alt="Modified `listobjA` by adding a new element to the list">

**Picture 7.** Modified `listobjA` by adding a new element to the list


**Step 5** - If we modify `listobjA` by assigning a list with different values to a variable `listobjA`, will `listobjA`'s ID change? For example, you can do something like this:
>`listobjA = [1,2,3,4,5]`

You can validate the result either by typing your code below and/or using Python Tutor.

In [40]:
# Solution, part 1 - code

listobjA = [1,2,3,44,5]
listobjA

[1, 2, 3, 44, 5]

In [41]:
# checking the type of the object listobjA

type(listobjA)

list

In [42]:
# validating an ID for the object listobjA

id(listobjA)

4445233672

You can compare the ID with the ID in the beginning of this notebook chapter. You will see that Python created new list object and pointed `listobjA` variable to this new list object which is a container for five integer objects. However, the `listobjB` still points to the same original object, nothing changed for the `listobjB`.

Here is a screen shot from Python Tutor to demonstrate the changes:

<img src='.\files\Task1_modifyA.jpg' width="600" height="600" alt="Modified `listobjA`">

**Picture 7.** Modified `listobjA`


Here is the [link](https://goo.gl/Fmu47e) to the screen above.

## More Python

In this section, we will cover additional Python functions: `map()` and Lambda function. They will come handy when we start manipulating DataFrame objects in the next section of this module, `pandas`. We will also learn how we can define a function when we do not know in advance how many attributes there will be when the function is called.

###  `map()` and `lambda` functions

It is important to note that in Python, functions can be passed as arguments to another function. Function `map()` usually takes a function as one of the parameters and applies it to the elements of an **iterable** object, which can be list, dictionary, or even a string.

Let's define **iterables** and **iterators** in Python. 

According to the Python's [Glossary of Terms](https://docs.python.org/3.6/glossary.html#term-iterable): **iterable** is "an object capable of returning its members one at a time." In other words, iterable is a Python object we can loop over. Lists, strings, tuples, dictionaries are all examples of iterables. 

An iterable object has a built-in method `__iter__()` which returns an [**iterator**](https://docs.python.org/3.6/glossary.html#term-iterator) object. An iterator in Python is "an object representing a stream of data", it enables iteration over each element within an iterable container. This object returns the data one element at a time with the method `__next__()`. 

We don't have to explicitly use any of these methods, but when we use, for example, a `for` statement to loop over a list, the `__next__()` method is called automatically to get each item from the iterator:

>`for element in [1, 2, 3]:    
>     print(element)`

See more examples in Python online documentation, chapter ["9.8. Iterators"](https://docs.python.org/3/tutorial/classes.html#iterators) of [The Python Tutorial](https://docs.python.org/3/tutorial/index.html) and section ["Iterators"](https://docs.python.org/dev/howto/functional.html#iterators) of ["Python Functional Programming HOWTO"](https://docs.python.org/dev/howto/functional.html).

If you have a list, you might need to loop through the list and apply a calculation defined by a function to each element of a list, or only to certain elements. In the example below, we will square each element of a list using Python function `map()`. 

Syntax of a function:
[`map(function, iterable)`](https://docs.python.org/3/library/functions.html#map). Let's review how this function is used.

In [28]:
# First, we create a custom square_it() function:

def square_it(x):
    return x * x

# Create a list

listA = [1, 2, 3, 4, 5, 6]

# Now we can use map() function:

list(map(square_it, listA))

[1, 4, 9, 16, 25, 36]

**NOTE:** Function `map()` returns an iterator object. In order to print out the values, we converted this iterator object to a list.

Function `square_it()` is very short function and can be replaced with the [**lambda function**](https://docs.python.org/dev/howto/functional.html#small-functions-and-the-lambda-expression). It is an anonymous function which does not have a name and thus cannot be reused elsewhere. It can be useful if, for example, we need to pass a simple one-line custom function to the other function. The lambda functions should only be used for very simple functions.

We can re-write the code above using the lambda function as follows:

In [30]:
list(map(lambda x: x * x, listA))

[1, 4, 9, 16, 25, 36]

In [31]:
# another example of using the lambda function:

list(map(lambda x: x + x, listA))

[2, 4, 6, 8, 10, 12]

### Defining arguments for a function

While creating a function, we might not know of all possible use cases when the function is going to be used. In order to handle a variable number of arguments during execution, we can define the function using `*args` and `**kwargs` syntax to define arguments.

The __single-asterisk `*` keyword__ is used when we need to use a list of arguments of variable length.

Here is an example of a function which will take a list of numbers and sum them up:

In [38]:
def my_sum(*args):
    a = 0
    for num in args:
        a += num
    print(a)

In [39]:
my_sum(4, 6, 10)

20


In [40]:
my_sum(3, 5.12, 45.78, 100, 123)

276.9


The __double-asterisk `**` keyword__ is used when we pass a keyword arguments, a dictionary.

In [32]:
def myfunc(**kwargs):
    for k,v in kwargs.items():
        print ("%s = %s" % (k, v))

# Using the function with one parameter:
myfunc(a=1)

a = 1


In [41]:
# Using 2 parameters

myfunc(b=2, a=1)

b = 2
a = 1


**NOTE:** The arguments don't have to be named `args` and `kwargs`. This is just a naming convention accepted by the community. However, the asterisk(s), `*` and `**`, is what defines the behaviour of the function call. To demonstrate, we can re-write the first function as follows:

In [42]:
def my_newsum(*listofargs):
    a = 0
    for num in listofargs:
        a += num
    print(a)
    
my_newsum(4, 6, 10)

20


You can read more on [Arbitrary Argument Lists](https://docs.python.org/3/tutorial/controlflow.html#arbitrary-argument-lists) in the Control Flow Tools section of [The Python Tutorial](https://docs.python.org/3/tutorial/index.html) documentation.

In [43]:
def catch_all(*args, **kwargs):
    print("args =", args)
    print("kwargs = ", kwargs)
    
catch_all(1, 2, 3, a=4, b=5)

args = (1, 2, 3)
kwargs =  {'a': 4, 'b': 5}


**End of Part 1.**


This notebook makes up one part of this module. Now that you have completed this part, please proceed to the next notebook in this module.

If you have any questions, please reach out to your peers using the discussion boards. If you and your peers are unable to come to a suitable conclusion, do not hesitate to reach out to your instructor on the designated discussion board.

## References

VanderPlas, Jake (2017). Basic Python Semantics: Variables and Objects. Everything Is an Object. In _A Whirlwind Tour of Python_. Free book, available at the website: [https://jakevdp.github.io/WhirlwindTourOfPython/03-semantics-variables.html#Everything-Is-an-Object](https://jakevdp.github.io/WhirlwindTourOfPython/03-semantics-variables.html#Everything-Is-an-Object)

Wikipedia. (2018). Object-oriented programming. Retrieved from [https://en.wikipedia.org/wiki/Object-oriented_programming](https://en.wikipedia.org/wiki/Object-oriented_programming)

Python Documentation. The Python Tutorial. (2018). Chapter 9, Classes. Retrieved from [https://docs.python.org/3.3/tutorial/classes.html](https://docs.python.org/3.3/tutorial/classes.html).

Python Tutor Website. (2018) Available at the following URL [http://pythontutor.com/live.html#mode=edit](http://pythontutor.com/live.html#mode=edit)

Python Documentation. (2018). Glossary, term _iterable_. Retrieved from [https://docs.python.org/3.6/glossary.html#term-iterable](https://docs.python.org/3.6/glossary.html#term-iterable)

Python Documentation. (2018). Glossary, term _iterator_. Retrieved from [https://docs.python.org/3.6/glossary.html#term-iterator](https://docs.python.org/3.6/glossary.html#term-iterator)

Python Documentation. The Python Tutorial. (2018). Section 9.8. Iterators. Retrieved from [https://docs.python.org/3/tutorial/classes.html#iterators](https://docs.python.org/3/tutorial/classes.html#iterators)

Kuchling, A.M. (2018). Iterators. In _Python Documentation. Python HOWTOs. Functional Programming HOWTO._ Retrieved from [https://docs.python.org/dev/howto/functional.html#iterators](https://docs.python.org/dev/howto/functional.html#iterators)

Python Documentation. The Python Standard Library. (2018). Chapter 2. Built-in Functions, `map()` function. Retrieved from [https://docs.python.org/3/library/functions.html#map](https://docs.python.org/3/library/functions.html#map)

Kuchling, A.M. (2018). Small functions and the lambda expression. In _Python Documentation. Python HOWTOs. Functional Programming HOWTO._ Retrieved from [https://docs.python.org/dev/howto/functional.html#small-functions-and-the-lambda-expression](https://docs.python.org/dev/howto/functional.html#small-functions-and-the-lambda-expression)

Python Documentation. The Python Tutorial. (2018). 4.7.3. Arbitrary Argument Lists. Retrieved from [https://docs.python.org/3/tutorial/controlflow.html#arbitrary-argument-lists](https://docs.python.org/3/tutorial/controlflow.html#arbitrary-argument-lists)